In [1]:
import os
import matplotlib as mpl
import matplotlib.pylab as plt
import itertools
import csv , codecs
import glob
from PIL import Image
import numpy as np
import pandas as pd
import openpyxl
from openpyxl import Workbook
from openpyxl import load_workbook
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
def warn(*args, **kwargs): pass
import warnings
warnings.warn = warn
from sklearn import decomposition
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, log_loss



In [2]:
data_dir = "./EEG_data/"

## CSV 파일 생성 함수

In [3]:
def EEG_data_csv(data_dir):
    
    # Kinds of Power 입력
    file_list = os.listdir(data_dir)
    file_list.sort()
    print("Kinds of Power" ,file_list)
       
    global kinds_of_power
    kinds_of_power =str(input( "Kinds of Power : "))
    
    excel_dir = "./EEG_data" + '/' + kinds_of_power
    excel_list = sorted(glob.glob(excel_dir+'/[!~]*.xlsx'))
    
    # EEG_csv_data 디렉토리 생성
    csv_dir = "./EEG_csv_data"
    if not os.path.isdir(csv_dir):
        os.mkdir(csv_dir)
    
    Abs_dir ="./EEG_csv_data/Abs"
    Relative_dir = "./EEG_csv_data/Relative"
    
    if not os.path.isdir(Abs_dir):
        os.mkdir(Abs_dir)
    if not os.path.isdir(Relative_dir):
        os.mkdir(Relative_dir)
    
    book = Workbook()
    sheet = book.active
    
    # x축,y축 생성
    for i in range (1,21) :
        if i < 10 :
            sheet['A'+str(i+1)] = 'S0'+str(i)
        if i == 10 :
            sheet['A'+str(i+1)] = 'S'+str(10)
        if i > 10 :
            sheet['A'+str(i+1)] = 'S'+str(i)
            
    for j in range (0,12):
        for i in range(1,33):
            sheet.cell(row=1, column=160*j+i+1).value = 'ACh' + str(i)
            sheet.cell(row=1, column=160*j+i+33).value = 'BCh' + str(i)
            sheet.cell(row=1, column=160*j+i+65).value = 'DCh' + str(i)
            sheet.cell(row=1, column=160*j+i+97).value = 'GCh' + str(i)
            sheet.cell(row=1, column=160*j+i+129).value = 'TCh' + str(i)

    # 파일을 순차적으로 열어서 셀 영역을 복사
    # j: 엑셀 파일의 수, k: 각 엑셀 파일 당 시트의 수, n: task의 수, l: subject의 수, m: ch1-ch32

    for j in range(0,5):
        wb = openpyxl.load_workbook(excel_list[j], data_only=True)
        wb_sheet = wb.sheetnames
        for k in range(0,12):
            source = wb[wb_sheet[k]]
            for l in range(2, 22):
                for m in range(2, 34):
                    sheet.cell(row=l,column=160*k+m+32*j).value = source.cell(row=l,column=m).value

    # EEG_csv_data 디렉토리에 csv 파일로 저장
    if kinds_of_power == "Abs":
        with open(Abs_dir + str('/') +'Abs_Merged.csv', 'w', newline="") as f:
            c = csv.writer(f)
            for r in sheet.rows:
                c.writerow([cell.value for cell in r])
    elif kinds_of_power == "Relative":
        with open(Relative_dir + str('/') +'Relative_Merged.csv', 'w', newline="") as f:
            c = csv.writer(f)
            for r in sheet.rows:
                c.writerow([cell.value for cell in r])

## CSV 파일에서 Pandas dataframe으로 가져오는 함수

In [4]:
def data_setting(task, eye , respiratory):
    # 성재가 만든 파일 불러오는 경로 입력 및 데이터 프레임으로 불러올것 
    ##  Abs, Relative 설정은 여기서 변경하기기
    csv_name = "./EEG_csv_data/Abs/Abs_Merged.csv"
    data = pd.read_csv(csv_name) 
    data_column = data.columns
    
    
    # 1 back
    if task == 0 : 
        if respiratory == 0 :
            data = data[data_column[1:161]]
            # 1-back open eye mouse (alpha, beta, delta, gamma, theta   ch(32*5))
            
            
            
        elif respiratory == 1 :
            data = data[data_column[161:321]]
            # 1-back open eye nose (alpha, beta, delta, gamma, theta   ch(32*5))
        elif respiratory == 2 :
            data = data[data_column[321:481]]
            # 1-back open eye mouse o2 (alpha, beta, delta, gamma, theta   ch(32*5))
    # 2 back
    elif task == 1 : 
        if respiratory == 0 :
            data = data[data_column[481:641]]
            # 2-back open eye mouse (alpha, beta, delta, gamma, theta   ch(32*5))
        elif respiratory == 1 :
            data= data[data_column[641:801]]
            # 2-back open eye nose (alpha, beta, delta, gamma, theta   ch(32*5))
        elif respiratory == 2 :
            data = data[data_column[801:961]]
            # 2-back open eye mouse o2 (alpha, beta, delta, gamma, theta   ch(32*5))
    #resting
    elif task == 2 :
        ## eye ==0 close eye
        if eye == 0 :
            if respiratory == 0 :
                data = data[data_column[961:1121]]
                # resting open eye mouse (alpha, beta, delta, gamma, theta   ch(32*5))
            elif respiratory == 1 :
                data = data[data_column[1121:1281]]
                # resting open eye nose (alpha, beta, delta, gamma, theta   ch(32*5))
            elif respiratory == 2 :
                data = data[data_column[1281:1441]]
                # resting open eye mouse o2 (alpha, beta, delta, gamma, theta   ch(32*5))
        ## eye ==1 open eye
        if eye == 1 : 
            if respiratory == 0 :
                data = data[data_column[1441:1601]]
                # resting close eye mouse (alpha, beta, delta, gamma, theta   ch(32*5))
            elif respiratory == 1 :
                data = data[data_column[1601:1761]]
                # resting close eye nose (alpha, beta, delta, gamma, theta   ch(32*5))
            elif respiratory == 2 :
                data = data[data_column[1761:1921]]
                # resting close eye mouse o2 (alpha, beta, delta, gamma, theta   ch(32*5))

    
    
    
    
    return data
                
                
# column indexing task condition

### task list

In [5]:
wb = openpyxl.load_workbook(data_dir+ str('/') +'Relative'+str('/')+'R_Alpha.xlsx')
wb_sheet = wb.sheetnames
wb_sheet

['B1_M',
 'B1_N',
 'B1_O2',
 'B2_M',
 'B2_N',
 'B2_O2',
 'Rest_EC_M',
 'Rest_EC_N',
 'Rest_EC_O2',
 'Rest_EO_M',
 'Rest_EO_N',
 'Rest_EO_O2']

### Kinds of Power 입력

In [6]:
EEG_data_csv(data_dir)

Kinds of Power ['Abs', 'Relative']


Kinds of Power :  Relative



### task 0 1 2 : 1-back  2-back  rest

### eye 0  1    : close eye     open eye

### respiratory 0 1 2 : mouse nose o2 



---

### SVM dataset 만드는 함수

In [7]:
def dataset_csv():
    
    excel_dir = "./EEG_data" + '/' + "Abs"
    excel_list = sorted(glob.glob(excel_dir+'/[!~]*.xlsx'))
    
    subject = int(input("subject :"))
    

    book = Workbook()
    sheet = book.active
    sheet.cell(row=1, column=161).value = 'target'
 
    # x축 생성   
    for i in range(1,33):
        sheet.cell(row=1, column=i).value = 'ACh' + str(i)
        sheet.cell(row=1, column=i+32).value = 'BCh' + str(i)
        sheet.cell(row=1, column=i+64).value = 'DCh' + str(i)
        sheet.cell(row=1, column=i+96).value = 'GCh' + str(i)
        sheet.cell(row=1, column=i+128).value = 'TCh' + str(i)
        
    # 파일을 순차적으로 열어서 셀 영역을 복사
    for j in range(0,5):
        wb = openpyxl.load_workbook(excel_list[j], data_only=True)
        wb_sheet = wb.sheetnames
        for k in range(0,12):
            source = wb[wb_sheet[k]]
            for l in range(2, 22):
                for m in range(2, 34):
                    sheet.cell(row=20*k+l,column=32*j+m-1).value = source.cell(row=l,column=m).value
        

    with open('./dataset_eeg.csv', 'w', newline="") as f:
        c = csv.writer(f)
        for r in sheet.rows:
            c.writerow([cell.value for cell in r])
            
    df = pd.read_csv('./dataset_eeg.csv')
    df[df =='                      NaN'] =np.nan
    df = df.fillna(0.0).astype('float64')
    pd.options.mode.chained_assignment = None

    df["target"][0*subject:1*subject] = "010"
    df["target"][1*subject:2*subject] = "011"
    df["target"][2*subject:3*subject] = "012"

    df["target"][3*subject:4*subject] = "110"
    df["target"][4*subject:5*subject] = "111"
    df["target"][5*subject:6*subject] = "112"

    df["target"][6*subject:7*subject] = "200"
    df["target"][7*subject:8*subject] = "201"
    df["target"][8*subject:9*subject] = "202"

    df["target"][9*subject:10*subject] = "210"
    df["target"][10*subject:11*subject] = "211"
    df["target"][11*subject:12*subject] = "212"
    df.astype('float64')
    df.to_csv('./dataset_eeg.csv')
    
    return df
            


In [8]:
eeg_data = dataset_csv()
eeg_data

subject : 20


C:\Users\hyg44\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


,ACh1,ACh2,ACh3,ACh4,ACh5,ACh6,ACh7,ACh8,ACh9,ACh10,...,TCh24,TCh25,TCh26,TCh27,TCh28,TCh29,TCh30,TCh31,TCh32,target
0,34.791231,43.380524,35.326932,33.483708,56.872380,80.370204,214.133509,336.869775,381.856303,273.700635,...,13.427554,8.834771,14.327464,14.443761,7.069934,2.461342,1.918468,3.426660,2.290185,010
1,21.332838,24.823823,7.732816,9.217822,9.089609,8.036041,15.609924,23.939677,45.026376,32.271230,...,13.955683,23.515145,33.013509,51.655861,26.336138,4.967240,4.443262,4.745022,4.208323,010
2,36.186846,30.996535,10.170486,9.205632,7.786313,11.105612,15.048448,23.943661,62.093379,51.073027,...,5.010105,15.810370,20.048230,32.001265,10.859633,1.375167,1.457062,1.650200,1.416968,010
3,165.706432,160.501820,53.815336,62.235116,55.541734,71.253082,91.474654,135.868921,134.070627,136.141713,...,16.361836,68.210183,54.308069,148.442087,63.703027,7.818248,6.864512,4.247089,4.067750,010
4,22.634142,32.687601,12.911995,12.740504,10.412052,13.157750,18.063993,45.134604,28.956921,28.032936,...,6.803263,10.276619,11.929224,19.722379,8.552856,2.337217,1.963548,1.735394,1.721120,010
5,9.904728,12.106190,5.481018,5.883165,6.943520,4.971094,11.073126,21.883341,28.940616,17.855794,...,5.201979,8.331477,9.298878,14.191144,6.611382,2.270006,1.609459,2.243871,1.464934,010
6,18.326832,18.923015,11.541991,11.720848,21.818624,16.801940,44.238363,45.513382,86.959537,96.298758,...,10.462003,6.948912,16.636356,19.600377,11.220615,4.065819,3.602533,4.167830,3.110982,010
7,24.845787,25.098427,9.968054,10.518397,11.365719,14.987010,34.950889,85.022078,65.631135,67.088813,...,13.238787,30.234171,23.676020,44.214361,45.812588,7.498409,4.584952,4.489311,4.936536,010
8,40.920388,38.523930,20.494293,16.728225,22.788190,17.159516,46.666191,66.393587,77.053292,70.261983,...,6.364726,11.050834,11.586133,21.515216,8.912505,2.377013,2.058713,2.536856,1.795944,010
9,38.508282,42.220113,8.909832,10.129484,11.125612,11.333431,59.035223,71.950390,127.573202,117.084965,...,4.425120,8.617088,9.871563,17.264892,6.362957,1.312544,1.445537,1.684411,1.398925,010


In [68]:
from sklearn.ensemble import RandomForestClassifier
X = eeg_data.iloc[:,0:-1]
X1=np.array(X)
Y = eeg_data['target']
Y1= np.array(Y)


X_train, X_test, y_train, y_test = train_test_split(X1, Y1, random_state=42)

rnd_clf = RandomForestClassifier(n_estimators=500, n_jobs=-1)
rnd_clf.fit(X_train, y_train)
for name, score in zip(X.columns.tolist(), rnd_clf.feature_importances_):
    print(name, score)

ACh1 0.007125009939699448
ACh2 0.0066040312269058325
ACh3 0.005417851993337901
ACh4 0.005260069675468172
ACh5 0.006607847763873775
ACh6 0.006476810732537043
ACh7 0.006146222326532957
ACh8 0.008518540232660412
ACh9 0.010051473457546312
ACh10 0.009924464439687981
ACh11 0.0054369653138406655
ACh12 0.005489040764163546
ACh13 0.007112046183820639
ACh14 0.0074504508606276654
ACh15 0.007594026776952496
ACh16 0.007271161757872422
ACh17 0.005254707376885013
ACh18 0.0
ACh19 0.00687483157088335
ACh20 0.0072218655221551855
ACh21 0.005901339689311269
ACh22 0.008387574312922207
ACh23 0.006910841725507946
ACh24 0.007273538560468258
ACh25 0.006197168570025275
ACh26 0.006837372233098253
ACh27 0.006706776254900155
ACh28 0.004889118668332929
ACh29 0.005366343423467266
ACh30 0.005484326354883441
ACh31 0.008444824962530255
ACh32 0.006327787909544056
BCh1 0.008763032135501803
BCh2 0.012310236708382532
BCh3 0.005674024032995736
BCh4 0.00510942753383425
BCh5 0.0070054673241253754
BCh6 0.00816151395952022
BCh7

# Feature importance로 상위 몇개의 feature을 가지고 만들어보기

In [74]:
from sklearn.ensemble import RandomForestClassifier

X = eeg_data.iloc[:,0:-1]
Y = eeg_data['target']
X1 = np.array(eeg_data.iloc[:,0:-1])
Y1 = np.array(eeg_data['target'])
X_train, X_test, y_train, y_test = train_test_split(X1, Y1, random_state=42)

rnd_clf = RandomForestClassifier(n_estimators=500, n_jobs=-1)
rnd_clf.fit(X_train, y_train)
print("0~155, (feature importance, 'ACh18,BCh18,DCh18,GCh18,TCh18'=0 )")
aa= int(input("top feature of number : " ))

channel = []
feature_list= list(zip(X.columns.tolist(),rnd_clf.feature_importances_))
feature_dict= dict(zip(X.columns.tolist(),rnd_clf.feature_importances_))
sorted_feature = sorted(feature_dict.values(), reverse=True)
top_feature = sorted_feature[0:aa]
choice_feature =[]


for i in range(len(feature_list)):
    for j in range(len(top_feature)):
        if feature_list[i][1] == top_feature[j]:
            channel.append(feature_list[i])
        else:
            continue


for i in range(len(channel)):
    choice_feature.append(channel[i][0])
    
#print(choice_feature)
choice_feature_df = eeg_data[choice_feature]
choice_feature_df.shape

0~155, (feature importance, 'ACh18,BCh18,DCh18,GCh18,TCh18'=0 )


top feature of number :  10


(240, 10)

In [75]:
channel

[('ACh9', 0.00958772630971731),
 ('BCh1', 0.009679175245212553),
 ('BCh2', 0.011823029733404752),
 ('DCh27', 0.009495437116557775),
 ('GCh1', 0.010021644099789288),
 ('GCh2', 0.017500973395773233),
 ('GCh21', 0.009810368014246685),
 ('GCh25', 0.009453013518309525),
 ('GCh26', 0.014292434010947059),
 ('GCh27', 0.013628117212742421)]

In [76]:
sorted_channel=[]
ranked_feature = sorted(top_feature,reverse=True) 
for i in range(len(channel)):
    for j in range(len(channel)):
        if channel[j][1] == ranked_feature[i]:
            sorted_channel.append(channel[j])
        else:
            continue
sorted_channel

[('GCh2', 0.017500973395773233),
 ('GCh26', 0.014292434010947059),
 ('GCh27', 0.013628117212742421),
 ('BCh2', 0.011823029733404752),
 ('GCh1', 0.010021644099789288),
 ('GCh21', 0.009810368014246685),
 ('BCh1', 0.009679175245212553),
 ('ACh9', 0.00958772630971731),
 ('DCh27', 0.009495437116557775),
 ('GCh25', 0.009453013518309525)]

In [77]:
len(channel)

10

In [78]:
X2 = np.array(choice_feature_df.iloc[:])

In [79]:
Y2= np.array(Y)

In [80]:
rnd_clf = RandomForestClassifier(n_estimators=500, n_jobs=-1)
rnd_clf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=-1, oob_score=False, random_state=None, verbose=0,
                       warm_start=False)

In [81]:
rnd_clf.score(X_train,y_train)

1.0

In [82]:
rnd_clf.score(X_test,y_test)

0.05

In [83]:

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
log_clf = LogisticRegression(solver='liblinear', random_state=42)
rnd_clf = RandomForestClassifier(n_estimators=10, random_state=42)
svm_clf = SVC(gamma='auto', probability=True, random_state=42)


##soft voting으로 바꿔줌줌
voting_clf = VotingClassifier(
    estimators=[('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)],
    voting='hard')
voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('lr',
                              LogisticRegression(C=1.0, class_weight=None,
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=100,
                                                 multi_class='warn',
                                                 n_jobs=None, penalty='l2',
                                                 random_state=42,
                                                 solver='liblinear', tol=0.0001,
                                                 verbose=0, warm_start=False)),
                             ('rf',
                              RandomForestClassifier(bootstrap=True,
                                                     class_weight=None,
                                                     criterion='gin...
                                        

In [84]:
voting_clf.score(X_train, y_train)

1.0

In [85]:
voting_clf.score(X_test, y_test)

0.016666666666666666

In [86]:

from sklearn.metrics import accuracy_score

for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

LogisticRegression 0.1
RandomForestClassifier 0.05
SVC 0.03333333333333333
VotingClassifier 0.016666666666666666


###  top feature importance selection으로 나눈 것은 결과가 좋지 못하다


---

---
---

# PCA 적용후 k-fold validation으로 SVM진행

In [87]:
X = eeg_data.iloc[:,0:-1]
X=np.array(X)

In [88]:
Y = eeg_data["target"]
Y = np.array(Y)

In [89]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVC



skf = StratifiedKFold(n_splits=10, shuffle=True)
skf.get_n_splits(X,Y)
all_scores = []


model = Pipeline((("scaler", StandardScaler()),("svm_clf", SVC(kernel='rbf',gamma='scale', decision_function_shape='ovo'))))

lda = LinearDiscriminantAnalysis(n_components=None)

lda = LinearDiscriminantAnalysis(n_components=11)
lda.fit(X,Y)

X_reduced_lda = lda.transform(X)


for train_index, test_index in skf.split(X_reduced_lda,Y):
    model.fit(X_reduced_lda[train_index], Y[train_index])
    val_mae = model.score(X_reduced_lda[test_index],Y[test_index])
    all_scores.append(val_mae)
    
print(all_scores)

[1.0, 0.75, 0.9583333333333334, 0.875, 0.875, 0.875, 0.9583333333333334, 0.9583333333333334, 0.9583333333333334, 0.9166666666666666]


In [90]:
print(np.mean(all_scores)*100,"%")

91.25 %


# PCA 적용후 k-fold validation으로 RandomForest진행

In [91]:
X = eeg_data.iloc[:,0:-1]
X=np.array(X)

In [92]:
Y = eeg_data["target"]
Y = np.array(Y)

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier

skf = StratifiedKFold(n_splits=10, shuffle=True)
skf.get_n_splits(X,Y)
all_scores = []

rnd_clf = RandomForestClassifier(n_estimators=500)
lda = LinearDiscriminantAnalysis(n_components=None)

lda = LinearDiscriminantAnalysis(n_components=11)
lda.fit(X,Y)

X_reduced_lda = lda.transform(X)


for train_index, test_index in skf.split(X_reduced_lda,Y):
    rnd_clf.fit(X_reduced_lda[train_index], Y[train_index])
    val_mae = rnd_clf.score(X_reduced_lda[test_index],Y[test_index])
    all_scores.append(val_mae)
    
print(all_scores)

In [ ]:
print(np.mean(all_scores)*100,"%")

# PCA 적용후 k-fold validation으로 Voting진행

In [ ]:
X = eeg_data.iloc[:,0:-1]
X=np.array(X)

In [ ]:
Y = eeg_data["target"]
Y = np.array(Y)

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC




log_clf = LogisticRegression(solver='liblinear', random_state=42)
rnd_clf = RandomForestClassifier(n_estimators=10, random_state=42)
svm_clf = SVC(gamma='auto', probability=True, random_state=42)

voting_clf = VotingClassifier(
    estimators=[('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)],
    voting='hard')


skf = StratifiedKFold(n_splits=10, shuffle=True)
skf.get_n_splits(X,Y)
all_scores = []

rnd_clf = RandomForestClassifier(n_estimators=500)
lda = LinearDiscriminantAnalysis(n_components=None)

lda = LinearDiscriminantAnalysis(n_components=11)
lda.fit(X,Y)

X_reduced_lda = lda.transform(X)


for train_index, test_index in skf.split(X_reduced_lda,Y):
    voting_clf.fit(X_reduced_lda[train_index], Y[train_index])
    val_mae = voting_clf.score(X_reduced_lda[test_index],Y[test_index])
    all_scores.append(val_mae)
    
print(all_scores)

In [ ]:
print(np.mean(all_scores)*100,"%")

In [ ]:
from sklearn.metrics import accuracy_score

for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
    clf.fit(X_reduced_lda[train_index], Y[train_index])
    y_pred = clf.predict(X_reduced_lda[test_index])
    print(clf.__class__.__name__, accuracy_score(Y[test_index], y_pred))

# RBF kernel을 사용하여 PCA를 시키고서는 SVM 적용해보기


In [42]:
from sklearn.decomposition import KernelPCA

rbf_pca = KernelPCA(kernel="rbf", gamma=0.0433, fit_inverse_transform=True)
X_reduced = rbf_pca.fit_transform(X)
X_reduced.shape

(240, 239)